# BetterWay 25. super로 부모 클래스를 초기화하자

- 기존에는 자식 클래스에서 부모 클래스의 `__init__` 메서드를 직접 호출하는 방법으로 부모클래스를 초기화 함

In [1]:
class MyBaseClass(object):
    def __init__(self, value):
        self.value = value
        
class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5)

- 위 방법은 간단한 계층 구조에는 잘 동작하지만, 많은 경우 제대로 동작하지 못함
- 클래스가 다중 상속 (보통은 피해야함... betterway26 참고)의 영향을 받는다면 슈퍼클래스의 `__init__` 메서드를 직접 호출하는 행위는 예기치 못한 동작을 일으킬 수 있다.

---
- 한가지 문제는 `__init__` 의 호출 순서가 모든 서브클래스에 걸쳐 명시되어 있지 않다는 점이다.
    - ex) 인스턴스의 value 필드로 연산을 수행하는 부모 클래스 두 개를 정의해보자.

In [2]:
class TimesTwo(object):
    def __init__(self):
        self.value *= 2
        
class PlusFive(object):
    def __init__(self):
        self.value += 5

In [3]:
# 다음 클래스는 한 가지 순서로 부모 클래스들을 정의한다.
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

- 이 클래스의 인스턴스를 생성하면 부모 클래스의 순서와 일치하는 결과가 만들어진다.

In [4]:
foo = OneWay(5)
print('First ordering is (5 *2) + 5 =', foo.value)

First ordering is (5 *2) + 5 = 15


- 다음은 같은 부모 클래스들을 다른 순서로 정의한 클래스다.

In [5]:
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

- 하지만 부모 클래스 생성자 `PlusFive.__init__, TimesTwo.__init__`을 이전과 같은 순서로 호출한다.
- 이 클래스의 동작은 부모 클래스를 정의한 순서와 일치하지 않는다.

In [6]:
bar = AnotherWay(5)
print('Second ordering still is', bar.value)

Second ordering still is 15


---

- 다른 문제는 다이아몬드 상속(diamond inheritance)이다.
    - 다이아몬드 상속 : 서브클래스가 계층 구조에서 가은 슈퍼클래스를 둔 서로 다른 두 클래스에서 상속받을 때 발생
- 다이아몬드 상속은 공통 슈퍼클래스의 `__init__` 메서드를 여러 번 실행하게 해서 예상치 못한 동작을 일으킴
- ex) MyBaseClass에서 상속받는 자식 클래스 두개를 정의해보자

In [7]:
class TimesFive(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self. value)
        self.value *= 5
        
class PlusTwo(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 2

- 다음으로 이 두 클래스 모두에서 상속받는 자식 클래스를 정의하여 MyBaseClass를 다이아몬드의 꼭대기로 만든다.

In [8]:
class ThisWay(TimesFive, PlusTwo):
    def __init__(self, value):
        TimesFive.__init__(self, value)
        PlusTwo.__init__(self, value)
        
foo = ThisWay(5)
print('Should be (5 * 5) + 2 = 27 but is', foo.value)

AttributeError: 'ThisWay' object has no attribute 'value'